In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
import lightgbm as lgb

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, BaggingRegressor, StackingRegressor, VotingRegressor, AdaBoostRegressor
from sklearn.experimental import enable_hist_gradient_boosting  # 이 줄은 아래 HistGradientBoostingRegressor를 사용하기 전에 필요합니다.
from sklearn.ensemble import HistGradientBoostingRegressor

/Users/bello/anaconda3/envs/ppap/lib/python3.9/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:15: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [2]:
###  INPUT ###
input_data = pd.read_csv('2023_smartFarm_AI_hackathon_dataset.csv') 
input_data.shape

(84840, 50)

In [3]:
df = input_data.drop(columns=['frmDist'])
df = df.sort_values(by='date')

# 데이터를 훈련 세트와 테스트 세트로 분할
'''
데이터를 훈련 세트와 테스트 세트로 나누는 데 사용됩니다. 
이는 모델의 성능을 평가하기 위해 데이터를 분리하는 일반적인 절차입니다.
'''
# 데이터셋을 data, target으로 변수분리
X = df[df.drop(columns=['outtrn_cumsum','HeatingEnergyUsage_cumsum']).columns]
Y = df[['outtrn_cumsum','HeatingEnergyUsage_cumsum']]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [10]:
class SimpleEnsemble(BaseEstimator, RegressorMixin):
    def __init__(self, models):
        self.models = models

    def fit(self, X, y):
        for model in self.models:
            model.fit(X, y)
        return self
    
    def predict(self, X):
        predictions = np.array([model.predict(X) for model in self.models])
        return np.mean(predictions, axis=0)

model_xgb = ('xgb', xgb.XGBRegressor())
GBoost = ('gboost', GradientBoostingRegressor())
LightGB = ('lgb', lgb.LGBMRegressor())
RF = ('rf', RandomForestRegressor())
ETR = ('etr', ExtraTreesRegressor())
DT = ('dt', DecisionTreeRegressor())
HGBR = ('hgbr', HistGradientBoostingRegressor())
Bagging = ('bagging', BaggingRegressor())

models = [GBoost, LightGB, RF, model_xgb, ETR, DT, HGBR, Bagging]

ensemble_model = SimpleEnsemble([estimator for _, estimator in models])

In [11]:
#### MODEL ####
model = StackingRegressor(estimators=models, final_estimator=ensemble_model)
model = MultiOutputRegressor(model)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002030 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9000
[LightGBM] [Info] Number of data points in the train set: 67872, number of used features: 39
[LightGBM] [Info] Start training from score 25537.923871


/Users/bello/anaconda3/envs/ppap/lib/python3.9/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/bello/anaconda3/envs/ppap/lib/python3.9/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/Users/bello/anaconda3/envs/ppap/lib/python3.9/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/Users/bello/anaconda3/envs/ppap/lib/python3.9/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

In [ ]:
# 성능 지표 계산
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2score = r2_score(y_test, y_pred)

### OUTPUT ###
print("RMSE:", rmse)
print("R2_score:", r2score)


RMSE: 24559.89362753205
R2_score: 0.9980704232243615
